## 任务6：多路召回+重排序case2

In [1]:
from datasets import load_dataset
import jieba
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import ndcg_score

In [2]:
data = load_dataset('C-MTEB/DuRetrieval')
qrels = load_dataset('C-MTEB/DuRetrieval-qrels')

In [3]:
data

DatasetDict({
    corpus: Dataset({
        features: ['id', 'text'],
        num_rows: 100001
    })
    queries: Dataset({
        features: ['id', 'text'],
        num_rows: 2000
    })
})

In [4]:
qrels

DatasetDict({
    dev: Dataset({
        features: ['qid', 'pid', 'score'],
        num_rows: 9839
    })
})

In [6]:
qrels_pd = pd.DataFrame(qrels["dev"])
qrels_dict = qrels_pd.groupby("qid")["pid"].apply(list).to_dict()

In [7]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel,OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np

In [8]:
corpus =[
    "今天 很开心",
    "明天 也 很开心",
    "Python is programming language"
]
corpus

['今天 很开心', '明天 也 很开心', 'Python is programming language']

In [9]:
corpus_split = [doc.lower().split() for doc in corpus]
corpus_split

[['今天', '很开心'],
 ['明天', '也', '很开心'],
 ['python', 'is', 'programming', 'language']]

In [10]:
dictionary=Dictionary(corpus_split)  ## 这个是将里面的每个value进行了去重，生成词典
for key,value in dictionary.items():
    print(key,value)

0 今天
1 很开心
2 也
3 明天
4 is
5 language
6 programming
7 python


In [11]:
bm25_model = OkapiBM25Model(dictionary=dictionary) # 相当于给模型喂一本词典

In [12]:


# dictionary.doc2bow：是一个方法，用来将文档表示为词袋的形式（Bag of Words），即将文档中的单词转换成整数索引，并计算它们的词频
# map(dictionary.doc2bow,corpus_split)：对corpus_split中的每个文档应用转换成词袋的表示形式
# list是将这些词袋表示的文档重新组合成一个列表
# bm25_model[...] 这部分代码使用了 Python 中的列表推导式，对 list(map(dictionary.doc2bow, corpus_split)) 中的每个文档应用了 bm25_model，即将文档转换为 Okapi BM25 模型的表示形式。
# bm25_model[list(map(dictionary.doc2bow, corpus_split))] 这行代码实际上是将整个语料库（由 corpus_split 表示）转换为 Okapi BM25 模型的表示形式，以便后续用于相似性计算和搜索

bm25_corpus = bm25_model[list(map(dictionary.doc2bow,corpus_split))] # 先转换成词袋，再转换成模型的表示形式
for i in bm25_corpus:
    print(i)

[(0, 0.6009713220776361), (1, 0.11268212288955677)]
[(1, 0.09577980445612325), (2, 0.5108256237659907), (3, 0.5108256237659907)]
[(4, 0.44419619457912235), (5, 0.44419619457912235), (6, 0.44419619457912235), (7, 0.44419619457912235)]


In [13]:
# bm25_corpus 是已经通过 Okapi BM25 模型处理过的语料库，其中每个文档都被表示为一个 Okapi BM25 模型表示形式。
# SparseMatrixSimilarity 是 Gensim 库中的类，用于构建一个稀疏矩阵相似性索引器。
# num_docs=len(corpus_split) 指定了语料库中文档的数量，这里使用了 len(corpus_split) 来获取文档数量。
# num_terms=len(dictionary) 指定了语料库中的唯一词汇数量，这里使用了 len(dictionary) 来获取词汇数量。
# normalize_queries=False 表示不对查询进行规范化，即不对查询向量进行归一化处理。
# normalize_documents=False 表示不对文档进行规范化，即不对文档向量进行归一化处理。
# 因此，整个行代码的含义是在基于 Okapi BM25 模型处理过的语料库上构建一个稀疏矩阵相似性索引器 bm25_index，以便后续用于执行相似性搜索操作。这个索引器可以用于计算查询与文档之间的相似性得分，从而找到与查询最匹配的文档。
# 生成了相似性索引器
bm25_index = SparseMatrixSimilarity(bm25_corpus,num_docs=len(corpus_split),num_terms=len(dictionary),normalize_queries=False,normalize_documents=False)


In [15]:
query = ["learn","今天"]
tfidf_model=TfidfModel(dictionary=dictionary,smartirs='bnn') # 使用之前的 字典，并bnn指定了对查询进行二进制加权
tfidf_query = tfidf_model[dictionary.doc2bow(query)] #将查询词转换成词袋的表示，并进行TF-IDF加权 
smiilarities = bm25_index[tfidf_query]  # 用相似性索引器 计算查询文本tfidf_query与语料库中的每个文档的相似性得分
print(smiilarities) # 打印出与每个文档的相似性得分
best_document=corpus_split[np.argmax(smiilarities)] # 找出相似性得分最大的，并且输出索引，最后查询出词
print(best_document)

[0.60097134 0.         0.        ]
['今天', '很开心']
